In [1]:
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
import os
import json
import re

from Codebase.DQN import DQN
from Codebase.EpsilonGreedyPolicy import EpsilonGreedyPolicy
from Codebase.ReplayMemory import ReplayMemory
from Codebase.train import run_episodes, train

import gym

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


In [2]:
with open(os.path.join("Codebase", "environment-info.json")) as data:
    env_data = json.load(data)

def Discrete(x):
    '''Helper function for data loading'''
    return True, int(x)

def Box(*args):
    '''Helper function for data loading'''
    return False, args
    
for game in env_data:
    print(game)
#     space = env_data[game]['action_space']
#     dim = eval(space)
#     if dim[0]:
#         print(dim)
#     else:
#         print("continuous boi")

Acrobot-v1
CartPole-v1
MountainCar-v0
MountainCarContinuous-v0
Breakout-v0
MontezumaRevenge-v0


In [6]:
# Let's run it!
num_episodes = 100
batch_size = 64
discount_factor = 0.9
learn_rate = 1e-3
memory = ReplayMemory(10000)

# game = "Acrobot-v1"
# game = "CartPole-v1"
game = "MountainCar-v0"
# game = "MountainCarContinuous-v0"
# game = "Breakout-v0"
# game = "MontezumaRevenge-v0"


env = gym.envs.make(game)


# Automatically find necessary DQN dimensions (only works for discrete actions atm) 
discrete_actions, action_no = eval(env_data[game]['action_space'])
if discrete_actions:
    output_size = action_no 
    
_, obs_no = eval(env_data[game]['observation_space'])
if len(obs_no) > 1:
    print("Convolution still has to be implemented...")
else:
    input_size = obs_no[0]


num_hidden = 256
seed = 42  # This is not randomly chosen

##!!!!!!!!!!!!! H A C K S !!!!!!!!!!!!!!!!##
env._max_episode_steps = 1000

# We will seed the algorithm (before initializing QNetwork!) for reproducibility
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
env.seed(seed)

Q_net = DQN(num_hidden, input_size, output_size)
policy = EpsilonGreedyPolicy(Q_net, 0.05)
episode_durations = run_episodes(train, Q_net, policy, memory, env, num_episodes, batch_size, discount_factor, learn_rate)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
 Episode 0 finished after 906 steps
 Episode 10 finished after 1000 steps
 Episode 20 finished after 604 steps
 Episode 30 finished after 723 steps
 Episode 40 finished after 343 steps
 Episode 50 finished after 272 steps
 Episode 60 finished after 624 steps
 Episode 70 finished after 1000 steps


KeyboardInterrupt: 

In [ ]:
# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

plt.plot(smooth(episode_durations, 10))
plt.title('Episode durations per episode')

In [5]:
??env.env